## Time Series Analysis of US Air Quality by State and County

### Part III: Review, clean, and combine dataset:

Author: Gem Ruby </br>
Date: April 2023

In [ ]:
## Import libraries
import pandas as pd
import os
import numpy as np
import requests
import warnings
warnings.filterwarnings("ignore")

# plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

# stats
from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#change directory
os.chdir('/content/drive/MyDrive/2022 - BrainStation/AirQuality_Capstone')

## Review AQI Components
- Understand the data included 
- Select one or more columns to include in the main dataset
- Review missing values and 0 values in relevant column

In [ ]:
#designate path:
components = '/content/drive/MyDrive/2022 - BrainStation/AirQuality_Capstone/Data/AQI Components/'
weather = '/content/drive/MyDrive/2022 - BrainStation/AirQuality_Capstone/Data/Meteorological/'

In [ ]:
#assign folder path
folder_path = components

# Get a list of file names in the folder for all components
AQI_file_names = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        AQI_file_names.append(filename)

In [ ]:
#list of all files for all AQI components
AQI_file_names

['so2.csv',
 'co.csv',
 'no2.csv',
 'pm2.5_frm.csv',
 'pm2.5_non.csv',
 'pm10.csv',
 'ozone.csv']

In [ ]:
#create dataframes 
def create_dataframes(folder_path):
    """
    Read all CSV files in a folder and create a pandas data frame for each file.

    Parameters:
    folder_path (str): Path to the folder containing CSV files.

    Returns:
    dict: A dictionary of data frames, where the keys are the file names (without the extension) and the values are the data frames.
    """
    # Create an empty dictionary to store the data frames
    dataframes = {}

    # Loop through all the CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            # Load the CSV file into a pandas dataframe
            file_path = os.path.join(folder_path, file_name)
            dataframe_name = os.path.splitext(file_name)[0]
            dataframe = pd.read_csv(file_path)
            # Store the dataframe in the dictionary with the name of the file (without the extension)
            dataframes[dataframe_name] = dataframe

    return dataframes

In [ ]:
#run dataframes
dataframes = create_dataframes(components)

In [ ]:
#assign the df
ozone_df = dataframes['ozone']
so2_df = dataframes['so2']
co_df = dataframes['co']
no2_df = dataframes['no2']
pm2_df = dataframes['pm2.5_frm']
pm2_non_df = dataframes['pm2.5_non']
pm10_df = dataframes['pm10']

In [ ]:
#List of dataframes and their names
Df_List = [ozone_df, so2_df, co_df, no2_df, pm2_df, pm2_non_df, pm10_df]
Df_names = ['ozone', 'so2', 'co', 'no2', 'pm2', 'pm2_non', 'pm10']

In [ ]:
#create identifer for all of the df containing: State_County_City_DateLocal
# Run a loop through "List_df"
for i, df in enumerate(Df_List):
    # Create a new column called 'ID' by combining existing columns
    df['ID'] = df['CBSA Name'].astype(str) + '_' + df['Date Local'].astype(str)
    # Replace any spaces in the 'ID' column with underscores
    df['ID'] = df['ID'].str.replace(' ', '_')
    # Update the data frame in the list with the new column
    Df_List[i] = df

    # Rename the 'AQI' column with the name of the data frame followed by 'AQI'
    new_aqi_col_name = Df_names[i] + '_AQI'
    df.rename(columns={'AQI': new_aqi_col_name}, inplace=True)

In [ ]:
#check one of the data frames:
ozone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2758537 entries, 0 to 2758536
Data columns (total 30 columns):
 #   Column               Dtype  
---  ------               -----  
 0   State Code           int64  
 1   County Code          int64  
 2   Site Num             int64  
 3   Parameter Code       int64  
 4   POC                  int64  
 5   Latitude             float64
 6   Longitude            float64
 7   Datum                object 
 8   Parameter Name       object 
 9   Sample Duration      object 
 10  Pollutant Standard   object 
 11  Date Local           object 
 12  Units of Measure     object 
 13  Event Type           object 
 14  Observation Count    int64  
 15  Observation Percent  float64
 16  Arithmetic Mean      float64
 17  1st Max Value        float64
 18  1st Max Hour         int64  
 19  ozone_AQI            int64  
 20  Method Code          float64
 21  Method Name          object 
 22  Local Site Name      object 
 23  Address              object 
 24

In [ ]:
ozone_df['CBSA Name'].value_counts()

Phoenix-Mesa-Scottsdale, AZ              71985
Riverside-San Bernardino-Ontario, CA     62968
New York-Newark-Jersey City, NY-NJ-PA    51081
Houston-The Woodlands-Sugar Land, TX     50510
Dallas-Fort Worth-Arlington, TX          47786
                                         ...  
The Dalles, OR                             424
Elko, NV                                   422
Walla Walla, WA                            419
Pocatello, ID                              365
Bend-Redmond, OR                           154
Name: CBSA Name, Length: 424, dtype: int64

In [ ]:
#check one of the dataframes
ozone_df.head(1)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,ID
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,NaN,-,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2021-11-11,"Daphne-Fairhope-Foley,_AL_2015-03-01"


In [ ]:
# Merge the dataframes on the 'ID' column
merged_df = pd.concat(Df_List, sort=False).groupby('ID', as_index=False).mean()

In [ ]:
#check merged info
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244363 entries, 0 to 1244362
Data columns (total 21 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ID                   1244363 non-null  object 
 1   State Code           1244363 non-null  float64
 2   County Code          1244363 non-null  float64
 3   Site Num             1244363 non-null  float64
 4   Parameter Code       1244363 non-null  float64
 5   POC                  1244363 non-null  float64
 6   Latitude             1244363 non-null  float64
 7   Longitude            1244363 non-null  float64
 8   Observation Count    1244363 non-null  float64
 9   Observation Percent  1244363 non-null  float64
 10  Arithmetic Mean      1244363 non-null  float64
 11  1st Max Value        1244363 non-null  float64
 12  1st Max Hour         1244363 non-null  float64
 13  ozone_AQI            892584 non-null   float64
 14  Method Code          1073313 non-null  float64
 15

In [ ]:
#check the new merged dataframe
merged_df.head(1)

,ID,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Observation Count,Observation Percent,...,1st Max Value,1st Max Hour,ozone_AQI,Method Code,so2_AQI,co_AQI,no2_AQI,pm2_AQI,pm2_non_AQI,pm10_AQI
0,"Aberdeen,_SD_2015-01-03",46.0,13.0,3.0,88101.0,2.0,45.4625,-98.486111,1.0,100.0,...,6.3,0.0,NaN,143.0,NaN,NaN,NaN,26.0,NaN,NaN


Combine all the AQI to the main dataframe with the actual AQI included.

In [ ]:
#read in the dataframe
cbsa = pd.read_csv('/content/drive/MyDrive/2022 - BrainStation/AirQuality_Capstone/Data/cbsa 2015-2022.csv')

In [ ]:
#check info
cbsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197150 entries, 0 to 1197149
Data columns (total 8 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   CBSA                       1197150 non-null  object
 1   CBSA Code                  1197150 non-null  int64 
 2   Date                       1197150 non-null  object
 3   AQI                        1197150 non-null  int64 
 4   Category                   1197150 non-null  object
 5   Defining Parameter         1197150 non-null  object
 6   Defining Site              1197150 non-null  object
 7   Number of Sites Reporting  1197150 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 73.1+ MB


In [ ]:
#check
cbsa.sample(5)

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,ID
428146,"Lawton, OK",30020,2017-08-16,41,Good,PM2.5,40-031-0651,1,"Lawton,_OK_2017-08-16"
133036,"San Jose-Sunnyvale-Santa Clara, CA",41940,2015-06-02,38,Good,Ozone,06-069-0003,7,"San_Jose-Sunnyvale-Santa_Clara,_CA_2015-06-02"
782313,"McAllen-Edinburg-Mission, TX",32580,2019-08-20,61,Moderate,PM2.5,48-215-0043,1,"McAllen-Edinburg-Mission,_TX_2019-08-20"
708676,"Cadillac, MI",15620,2019-09-05,29,Good,Ozone,26-113-0001,2,"Cadillac,_MI_2019-09-05"
540228,"Charleston, WV",16620,2018-07-15,36,Good,Ozone,54-039-0020,1,"Charleston,_WV_2018-07-15"


In [ ]:
#create 'ID' column in cbsa
cbsa['ID'] = cbsa['CBSA'].astype(str) + '_' + cbsa['Date'].astype(str)

# Replace any spaces in the 'ID' column with underscores
cbsa['ID'] = cbsa['ID'].str.replace(' ', '_')

#check cbsa 
cbsa.head(1)

,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,ID
0,"Aberdeen, SD",10100,2015-01-03,26,Good,PM2.5,46-013-0003,1,"Aberdeen,_SD_2015-01-03"


In [ ]:
#concatenate the final_df
cbsa_df = pd.merge(merged_df, cbsa, on='ID')

#check final_df
cbsa_df.head(1)

,ID,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Observation Count,Observation Percent,...,pm2_non_AQI,pm10_AQI,CBSA,CBSA Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,"Aberdeen,_SD_2015-01-03",46.0,13.0,3.0,88101.0,2.0,45.4625,-98.486111,1.0,100.0,...,NaN,NaN,"Aberdeen, SD",10100,2015-01-03,26,Good,PM2.5,46-013-0003,1


In [ ]:
#create final df
final_df = cbsa_df[['CBSA','CBSA Code','State Code','County Code','Date','AQI','ozone_AQI', 'so2_AQI','co_AQI','no2_AQI','pm2_AQI','pm2_non_AQI','pm10_AQI']]

In [ ]:
#confirm final df 
final_df

,CBSA,CBSA Code,State Code,County Code,Date,AQI,ozone_AQI,so2_AQI,co_AQI,no2_AQI,pm2_AQI,pm2_non_AQI,pm10_AQI
0,"Aberdeen, SD",10100,46.0,13.0,2015-01-03,26,NaN,NaN,NaN,NaN,26.0,NaN,NaN
1,"Aberdeen, SD",10100,46.0,13.0,2015-01-06,28,NaN,NaN,NaN,NaN,28.0,NaN,8.0
2,"Aberdeen, SD",10100,46.0,13.0,2015-01-09,25,NaN,NaN,NaN,NaN,25.0,NaN,NaN
3,"Aberdeen, SD",10100,46.0,13.0,2015-01-12,26,NaN,NaN,NaN,NaN,25.0,NaN,6.0
4,"Aberdeen, SD",10100,46.0,13.0,2015-01-15,21,NaN,NaN,NaN,NaN,21.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197145,"Yuma, AZ",49740,4.0,27.0,2021-12-27,34,33.0,NaN,NaN,NaN,34.0,NaN,29.0
1197146,"Yuma, AZ",49740,4.0,27.0,2021-12-28,34,34.0,NaN,NaN,NaN,19.0,NaN,16.0
1197147,"Yuma, AZ",49740,4.0,27.0,2021-12-29,35,35.0,NaN,NaN,NaN,23.0,NaN,22.0
1197148,"Yuma, AZ",49740,4.0,27.0,2021-12-30,27,27.0,NaN,NaN,NaN,14.0,NaN,11.0


In [ ]:
#check NaN values
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1197150 entries, 0 to 1197149
Data columns (total 13 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   CBSA         1197150 non-null  object 
 1   CBSA Code    1197150 non-null  int64  
 2   State Code   1197150 non-null  float64
 3   County Code  1197150 non-null  float64
 4   Date         1197150 non-null  object 
 5   AQI          1197150 non-null  int64  
 6   ozone_AQI    887844 non-null   float64
 7   so2_AQI      485271 non-null   float64
 8   co_AQI       313096 non-null   float64
 9   no2_AQI      387512 non-null   float64
 10  pm2_AQI      734818 non-null   float64
 11  pm2_non_AQI  436449 non-null   float64
 12  pm10_AQI     408221 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 127.9+ MB


In [ ]:
#confirm CBSA and CBSA code have the same value counts
final_df[['CBSA','CBSA Code']].value_counts()

CBSA                      CBSA Code
Yuma, AZ                  49740        2557
Missoula, MT              33540        2557
New Orleans-Metairie, LA  35380        2557
Columbia, SC              17900        2557
New Haven-Milford, CT     35300        2557
                                       ... 
Jonesboro, AR             27860         139
Crawfordsville, IN        18820         118
Coos Bay, OR              18300         107
Lexington, NE             30420          71
Beckley, WV               13220          25
Length: 539, dtype: int64

In [ ]:
# Calculate the percentage of NaN values in each column
nan_percentages = final_df.isna().mean() * 100

# Create a new dataframe
nan_df = pd.DataFrame({'Column': nan_percentages.index, 'NaN Percentage': nan_percentages.values})

#review new dataframe
nan_df

,Column,NaN Percentage
0,CBSA,0.000000
1,CBSA Code,0.000000
2,State Code,0.000000
3,County Code,0.000000
4,Date,0.000000
5,AQI,0.000000
6,ozone_AQI,25.836863
7,so2_AQI,59.464478
8,co_AQI,73.846552
9,no2_AQI,67.630456


In [ ]:
#read in the dataframe
county = pd.read_csv('/content/drive/MyDrive/2022 - BrainStation/AirQuality_Capstone/Data/county 2015-2022.csv')

In [ ]:
#review the dataframe
county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2271501 entries, 0 to 2271500
Data columns (total 10 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   State Name                 object
 1   county Name                object
 2   State Code                 int64 
 3   County Code                int64 
 4   Date                       object
 5   AQI                        int64 
 6   Category                   object
 7   Defining Parameter         object
 8   Defining Site              object
 9   Number of Sites Reporting  int64 
dtypes: int64(4), object(6)
memory usage: 173.3+ MB


Conclusion:

Due to the large number of missing AQI components, we will be forgoing to include this dataset. Note that almost all of the dataset have less than 50% of data available. Also, based on the dataset and my original research, there was a change in parameter measurement by testing facilities. 
